# INTRO TENSOR FLOW

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
# from MNISTData import MNISTData

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# mnist = MNISTData(train_dir='MNIST_data', one_hot=True)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

mnist.train.__class__

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [3]:
# Tensor Flow
import tensorflow as tf

In [4]:
# set parameters
learning_rate = 0.01 # how fast to update the weight
training_iteration = 30
batch_size = 100
display_step = 2

In [5]:
# TF graph input
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])


# Create a model

# Set model weights
W = tf.Variable(tf.zeros([784, 10])) # prob data flows
b = tf.Variable(tf.zeros([10])) # bias

with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x,W) + b) # softmax
    
# Add summary ops to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [6]:
# more name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)
    
with tf.name_scope("train") as scope:
    # gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
    
# init variables
init = tf.initialize_all_variables()

# merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Change this to a location on your computer
    summary_writer = tf.summary.FileWriter('/Users/lesly/Documents/workspace/pythondev/tensorflow_test/logs', graph_def=sess.graph_def)

    # training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch

            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print "Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost)

    print "Tuning completed!"
    
    # test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))

    # Calculate accuracy
    # accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    # print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
    accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))
    print("Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y: mnist.test.labels}))

Iteration: 0001 cost= 30.921844241
Iteration: 0003 cost= 20.963844898
Iteration: 0005 cost= 20.156273942
Iteration: 0007 cost= 19.623376702
Iteration: 0009 cost= 19.303755054
Iteration: 0011 cost= 19.159618158
Iteration: 0013 cost= 18.831904376
Iteration: 0015 cost= 18.808318597
Iteration: 0017 cost= 18.574168326
Iteration: 0019 cost= 18.520672861
Iteration: 0021 cost= 18.346288888
Iteration: 0023 cost= 18.416132818
Iteration: 0025 cost= 18.138371902
Iteration: 0027 cost= 18.174869237
Iteration: 0029 cost= 18.122377126
Tuning completed!
('Accuracy:', 0.92320001)
